# Create synthetic mouse phenome data

Create a synthetic version of the mouse phenomes from the original experiment, which are available after running `01_create_phenome_training_data.ipynb`. To run this notebook, you will need an API key from the Gretel console,  at https://console.gretel.cloud.

In [ ]:
%%capture
!pip install -U gretel-client

In [ ]:
# Specify your Gretel API key

from getpass import getpass
import pandas as pd
from gretel_client import configure_session, ClientConfig

pd.set_option('max_colwidth', None)

configure_session(ClientConfig(api_key=getpass(prompt="Enter Gretel API key"), 
                               endpoint="https://api-dev.gretel.cloud"))

                            

## Configure model hyper parameters
Load the default configuration template. This template will work well for most datasets. View other templates at https://github.com/gretelai/gretel-blueprints/tree/main/config_templates/gretel/synthetics

In [ ]:
import json
from smart_open import open
import yaml

with open("https://raw.githubusercontent.com/gretelai/gretel-blueprints/main/config_templates/gretel/synthetics/default.yml", 'r') as stream:
    config = yaml.safe_load(stream)

# Optimize parameters for complex dataset
config['models'][0]['synthetics']['params']['epochs'] = 200
config['models'][0]['synthetics']['params']['vocab_size'] = 0
config['models'][0]['synthetics']['params']['rnn_units'] = 640
config['models'][0]['synthetics']['params']['reset_states'] = False
config['models'][0]['synthetics']['params']['learning_rate'] = 0.001
config['models'][0]['synthetics']['params']['dropout_rate'] = 0.4312
config['models'][0]['synthetics']['params']['gen_temp'] = 1.003
config['models'][0]['synthetics']['privacy_filters']['similarity'] = None

print(json.dumps(config, indent=2))

## Train the synthetic model
In this step, we will task the worker running in the Gretel cloud, or locally, to train a synthetic model on the source dataset.

In [ ]:
# Get the location of the phenome training data

import os
import pathlib

base_path = pathlib.Path(os.getcwd().replace("/synthetics", ""))
data_path = base_path / 'data'

In [ ]:
# Define a function to submit a new model for a specific phenome batch dataset

def create_model(batch_num):
    seconds = int(time.time())
    project_name = "Training phenomes" + str(seconds)
    project = create_project(display_name=project_name)
    batchfile = "pheno_batch" + str(batch_num) + ".csv"
    trainpath = str(data_path / batchfile)
    model = project.create_model_obj(model_config=config)
    model.data_source = trainpath
    model.submit(upload_data_source=True)  
    return(model)

In [ ]:
# Submit all the phenome batches to train in parallel; poll for completion

from gretel_client.helpers import poll
from gretel_client import create_project
import time

# Create a model for each batch
models = []
for i in range(7):
    model = create_model(i)
    models.append(model)

# Poll for completion. Resubmit errors.
training = True
while training:
    time.sleep(60)
    training = False
    print()
    for i in range(7):
        model = models[i]
        model._poll_job_endpoint()
        status = model.__dict__['_data']['model']['status']
        print("Batch " + str(i) + " has status: " + status)
        if ((status == "active") or (status == "pending")):
            training = True
        if status == "error":
            model = create_model(i)
            models[i] = model
            training = True           

# Now that models are complete, get each batches Synthetic Quality Score (SQS)            
batch = 0
print()
for model in models:
    model._poll_job_endpoint()
    status = model.__dict__['_data']['model']['status']
    if status == "error":
        print("Batch " + str(batch) + " ended with error")
    else:
        report = model.peek_report()
        sqs = report['synthetic_data_quality_score']['score']
        label = "Moderate"
        if sqs >= 80:
            label = "Excellent"
        elif sqs >= 60:
            label = "Good"
        print("Batch " + str(batch) + " completes with SQS: " + label + " (" + str(sqs) + ")")
    batch += 1

In [ ]:
# Read the original phenome set

filename = "phenome_alldata.csv"
filepath = data_path / filename
phenome_orig = pd.read_csv(filepath)

In [ ]:
# Merge the synthetic batches into one dataframe
# First gather the synthetic data for each batch

synth_batches = []
for i in range(7):
    model = models[i]
    synth = pd.read_csv(model.get_artifact_link("data_preview"), compression='gzip')
    synth_batches.append(synth)


In [ ]:
# Merge batch 0 and 1 on common field sacweight
synth_batches[0]['g'] = synth_batches[0].groupby('sacweight').cumcount()
synth_batches[1]['g'] = synth_batches[1].groupby('sacweight').cumcount()
synth_allbatches = pd.merge(synth_batches[0],synth_batches[1],on=["sacweight", 'g'],how='left').drop('g', axis=1)

In [ ]:
# Now merge in batch 2 on common fields SW16, SW20, SW17
synth_allbatches['g'] = synth_allbatches.groupby(['SW16','SW20', 'SW17']).cumcount()
synth_batches[2]['g'] = synth_batches[2].groupby(['SW16', 'SW20', 'SW17']).cumcount()
synth_allbatches = pd.merge(synth_allbatches,synth_batches[2],on=['SW16', 'SW20', 'SW17', 'g'],how='left').drop('g', axis=1)

In [ ]:
# Now merge in batches 3 
synth_allbatches = pd.concat([synth_allbatches, synth_batches[3]], axis=1)

In [ ]:
# Now merge in batch 4 using common fields 'methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10'
synth_allbatches['g'] = synth_allbatches.groupby(['methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10']).cumcount()
synth_batches[4]['g'] = synth_batches[4].groupby(['methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10']).cumcount()
synth_allbatches = pd.merge(synth_allbatches,synth_batches[4],on=['methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10', 'g'],how='left').drop('g', axis=1)

In [ ]:
# Now merge in batch 5 using common fields 'methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10'
synth_allbatches['g'] = synth_allbatches.groupby(['methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10']).cumcount()
synth_batches[5]['g'] = synth_batches[5].groupby(['methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10']).cumcount()
synth_allbatches = pd.merge(synth_allbatches,synth_batches[5],on=['methcage12', 'methcage9', 'methcage11', 'methcage8', 'methcage7', 'methcage10', 'g'],how='left').drop('g', axis=1)

In [ ]:
# Now merge in batches 6
synth_allbatches = pd.concat([synth_allbatches, synth_batches[6]], axis=1)

In [ ]:
synth_allbatches.head()

In [ ]:
# Save off complete synthetic data
filename = "phenome_alldata_synth.csv"
filepath = data_path / filename
synth_allbatches.to_csv(filepath, index=False, header=True)

In [ ]:
# Generate report that shows the statistical performance between the training and synthetic data
# Use the synthetic batch that includes abBMD

import IPython
from smart_open import open

# Change batch_num to any value between 0 and 6 to view performance report for other batches
batch_num = 0
IPython.display.HTML(data=open(models[0].get_artifact_link("report")).read())
